In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [ ]:
sheets = [ pd.read_excel( 'RomanUrduToxicity.xlsx', 'Sheet1' ), pd.read_excel( 'RomanUrduToxicity.xlsx', 'Sheet2' ) ]
dforiginal = pd.concat( sheets )
dforiginal.reset_index( drop=True, inplace=True )
dforiginal.Comment = dforiginal.Comment.astype( 'str' )
del sheets
dforiginal.shape

In [ ]:
dfs = []
dfs.append( pd.read_csv( './Results/NB.csv' ) )
dfs.append( pd.read_csv( './Results/RF.csv' ) )
dfs.append( pd.read_csv( './Results/LR.csv' ) )
dfs.append( pd.read_csv( './Results/SVM.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_George.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked.csv' ) )

dfs.append( pd.read_csv( './Results/CNN_George_glove.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P_glove.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU_glove.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM_glove.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_glove.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked_glove.csv' ) )

dfs.append( pd.read_csv( './Results/CNN_George_w2v_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P_w2v_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU_w2v_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM_w2v_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_w2v_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked_w2v_cbow.csv' ) )

dfs.append( pd.read_csv( './Results/CNN_George_w2v_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P_w2v_sg.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU_w2v_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM_w2v_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_w2v_sg.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked_w2v_sg.csv' ) )

dfs.append( pd.read_csv( './Results/CNN_George_ft_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P_ft_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU_ft_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM_ft_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_ft_cbow.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked_ft_cbow.csv' ) )

dfs.append( pd.read_csv( './Results/CNN_George_ft_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_P_ft_sg.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_GRU_ft_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BLSTM_ft_sg.csv' ) )
dfs.append( pd.read_csv( './Results/BGRU_ft_sg.csv' ) )
dfs.append( pd.read_csv( './Results/CNN_Tweaked_ft_sg.csv' ) )
len(dfs)

In [ ]:
dfs[0].head()

In [ ]:
modelnames = [ 'NB', 'RF', 'LR', 'SVM', 'CNN_George', 'BGRU_P', 'CNN_GRU', 'BLSTM', 'BGRU', 'CNN_Tweaked',
             'CNN_George_glove', 'BGRU_P_glove', 'CNN_GRU_glove', 'BLSTM_glove', 'BGRU_glove', 'CNN_Tweaked_glove',
             'CNN_George_w2v_cbow', 'BGRU_P_w2v_cbow', 'CNN_GRU_w2v_cbow', 'BLSTM_w2v_cbow', 'BGRU_w2v_cbow', 'CNN_Tweaked_w2v_cbow',
             'CNN_George_w2v_sg', 'BGRU_P_w2v_sg', 'CNN_GRU_w2v_sg', 'BLSTM_w2v_sg', 'BGRU_w2v_sg', 'CNN_Tweaked_w2v_sg',
             'CNN_George_ft_cbow', 'BGRU_P_ft_cbow', 'CNN_GRU_ft_cbow', 'BLSTM_ft_cbow', 'BGRU_ft_cbow', 'CNN_Tweaked_ft_cbow',
             'CNN_George_ft_sg', 'BGRU_P_ft_sg', 'CNN_GRU_ft_sg', 'BLSTM_ft_sg', 'BGRU_ft_sg', 'CNN_Tweaked_ft_sg']
mlmodels = [ 'NB', 'RF', 'LR', 'SVM' ]
deepmodels = [ m for m in modelnames if m not in mlmodels ]
ml_bestdeep = [ 'NB', 'RF', 'LR', 'SVM', 'CNN_George_ft_sg', 'BGRU_P_ft_sg', 'CNN_GRU_ft_sg', 'BLSTM_ft_sg', 'BGRU_ft_sg', 'CNN_Tweaked_ft_sg' ]
bestdeep = [ 'CNN_George_ft_sg', 'BGRU_P_ft_sg', 'CNN_GRU_ft_sg', 'BLSTM_ft_sg', 'BGRU_ft_sg', 'CNN_Tweaked_ft_sg' ]

In [ ]:
def maj_vote( dataframe, alabel, models ):
    accuracy, precision, recall, f1, yy_true, yy_pred = [], [], [], [], [], []
    
    dataframe[ 'maj_vote' ] = dataframe[ models ].sum( axis=1 )
    dataframe[ 'maj_vote_res' ] = dataframe[ 'maj_vote' ].apply( lambda x:  1 if x/len(models) >= 0.5 else 0  )
    
    skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
    skf.get_n_splits( dforiginal.Comment, dataframe[ alabel ] )
    
    i = 1
    for train_index, test_index in skf.split( dforiginal.Comment, dataframe[ alabel ] ):
        ytest = dataframe.loc[ test_index ][ 'comment_label' ].values
        testpredictions = dataframe.loc[ test_index ][ 'maj_vote_res' ].values
        
        accuracy.append( accuracy_score( ytest, testpredictions ) )
        precision.append( precision_score( ytest, testpredictions ) )
        recall.append( recall_score( ytest, testpredictions ) )
        f1.append( f1_score( ytest, testpredictions ) )
        yy_true.extend( ytest.tolist() )
        yy_pred.extend( testpredictions.tolist() )

        i = i + 1
    
    return accuracy, precision, recall, f1, yy_true, yy_pred

def avg_prob( dataframe, alabel, models ):
    accuracy, precision, recall, f1 = [], [], [], []
    
    dataframe[ 'avg_prob' ] = dataframe[ models ].mean( axis=1 )
    dataframe[ 'avg_prob_res' ] = dataframe[ 'avg_prob' ].apply( lambda x:  1 if x >= 0.5 else 0  )
    
    skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
    skf.get_n_splits( dforiginal.Comment, dataframe[ alabel ] )
    
    i = 1
    for train_index, test_index in skf.split( dforiginal.Comment, dataframe[ alabel ] ):
        ytest = dataframe.loc[ test_index ][ 'comment_label' ].values
        testpredictions = dataframe.loc[ test_index ][ 'avg_prob_res' ].values
        
        accuracy.append( accuracy_score( ytest, testpredictions ) )
        precision.append( precision_score( ytest, testpredictions ) )
        recall.append( recall_score( ytest, testpredictions ) )
        f1.append( f1_score( ytest, testpredictions ) )

        i = i + 1
    return accuracy, precision, recall, f1, dataframe[ 'avg_prob' ]

In [ ]:
dflabels = pd.DataFrame()
dflabels[ 'comment_indices' ] = dfs[0][ 'comment_indices' ]
#dflabels[ 'comment_text' ] = dfs[0][ 'comment_text' ]
dflabels[ 'comment_label' ] = dfs[0][ 'comment_label' ]

for i in range( len( dfs ) ):
    dflabels[ modelnames[i] ] = dfs[i][ 'comment_predicted' ]
dflabels = dflabels.sort_values(by='comment_indices').reset_index( drop=True )
dflabels.head()

In [ ]:
dfprobs = pd.DataFrame()
dfprobs[ 'comment_indices' ] = dfs[0][ 'comment_indices' ]
#dfprobs[ 'comment_text' ] = dfs[0][ 'comment_text' ]
dfprobs[ 'comment_label' ] = dfs[0][ 'comment_label' ]

for i in range( len( dfs ) ):
    dfprobs[ modelnames[i] ] = dfs[i][ 'comment_prob' ]
dfprobs = dfprobs.sort_values(by='comment_indices').reset_index( drop=True )
dfprobs.head()

In [ ]:
def ensemble_results( filename, mode, name, accuracy, precision, recall, f1 ):
    file = open( filename, mode=mode )
    file.write( name )
    file.write( ',' )
    file.write( str(np.mean( accuracy ))[:7] + '+-' + str(np.std( accuracy ))[:6] )
    file.write( ',' )
    file.write( str(np.mean( precision ))[:7] + '+-' + str(np.std( precision ))[:6] )
    file.write( ',' )
    file.write( str(np.mean( recall ))[:7] + '+-' + str(np.std( recall ))[:6] )
    file.write( ',' )
    file.write( str(np.mean( f1 ))[:7] + '+-' + str(np.std( f1 ))[:6] )
    file.write( '\n' )
    file.close()
    return

In [ ]:
accuracy, precision, recall, f1, yy_true, yy_pred = maj_vote( dflabels, 'comment_label', modelnames )
ensemble_results( './Results/ensemble.csv', 'w', 'All (maj_vote)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, prr = avg_prob( dfprobs, 'comment_label', modelnames )
ensemble_results( './Results/ensemble.csv', 'a', 'All (avg_prob)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, yy_true, yy_pred = maj_vote( dflabels, 'comment_label', deepmodels )
ensemble_results( './Results/ensemble.csv', 'a', 'All Deep (maj_vote)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, prr = avg_prob( dfprobs, 'comment_label', deepmodels )
ensemble_results( './Results/ensemble.csv', 'a', 'All Deep (avg_prob)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, yy_true, yy_pred = maj_vote( dflabels, 'comment_label', mlmodels )
ensemble_results( './Results/ensemble.csv', 'a', 'All ML (maj_vote)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, prr = avg_prob( dfprobs, 'comment_label', mlmodels )
ensemble_results( './Results/ensemble.csv', 'a', 'All ML (avg_prob)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, yy_true, yy_pred = maj_vote( dflabels, 'comment_label', ml_bestdeep )
ensemble_results( './Results/ensemble.csv', 'a', 'ML + Best Deep (maj_vote)', accuracy, precision, recall, f1 )

In [ ]:
c = confusion_matrix( yy_true, yy_pred )
print( np.rot90(np.rot90(c)), '\n\n' )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, prr = avg_prob( dfprobs, 'comment_label', ml_bestdeep )
ensemble_results( './Results/ensemble.csv', 'a', 'ML + Best Deep (avg_prob)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, yy_true, yy_pred = maj_vote( dflabels, 'comment_label', bestdeep )
ensemble_results( './Results/ensemble.csv', 'a', 'Best Deep (maj_vote)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )

In [ ]:
accuracy, precision, recall, f1, prr = avg_prob( dfprobs, 'comment_label', bestdeep )
ensemble_results( './Results/ensemble.csv', 'a', 'Best Deep (avg_prob)', accuracy, precision, recall, f1 )

In [ ]:
print( ' Accuracy' )
print( [ '{:0.4f}'.format( x ) for x in accuracy ], np.mean( accuracy ), '+-', np.std( accuracy ), '\n' )

print( ' Precision' )
print( [ '{:0.4f}'.format( x ) for x in precision ], np.mean( precision ), '+-', np.std( precision ), '\n' )

print( ' Recall' )
print( [ '{:0.4f}'.format( x ) for x in recall ], np.mean( recall ), '+-', np.std( recall ), '\n' )

print( ' F1' )
print( [ '{:0.4f}'.format( x ) for x in f1 ], np.mean( f1 ), '+-', np.std( f1 ) )